In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

import mlflow
from mlflow.models import infer_signature

In [2]:
# load the dataset
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow-example/refs/heads/master/wine-quality.csv")

# Train Test Split
train,test = train_test_split(data, test_size=0.5, random_state=42)
train_x = train.drop("quality", axis = 1).values
train_y = train[['quality']].values.ravel()
test_x = test.drop("quality", axis = 1).values
test_y = test[['quality']].values.ravel()

# Train Valid Split
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)


In [3]:
singature = infer_signature(train_x, train_y)
singature

inputs: 
  [Tensor('float64', (-1, 11))]
outputs: 
  [Tensor('int64', (-1,))]
params: 
  None

In [4]:
### ANN Model
class ANNModel(nn.Module):
    def __init__(self, input_dim, mean, var):
        super(ANNModel, self).__init__()
        self.norm = nn.BatchNorm1d(input_dim)

        for param in self.norm.parameters():
            param.requires_grad = False
        self.norm.running_mean = mean
        self.norm.running_var = var

        self.fc1 = nn.Linear(input_dim, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = self.norm(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    mean=torch.tensor(np.mean(train_x,axis=0), dtype=torch.float32)
    var=torch.tensor(np.var(train_x,axis=0), dtype=torch.float32)

    # Convert NumPy arrays to PyTorch tensors
    train_x, train_y = torch.tensor(train_x, dtype=torch.float32), torch.tensor(train_y, dtype=torch.float32).view(-1, 1)
    valid_x, valid_y = torch.tensor(valid_x, dtype=torch.float32), torch.tensor(valid_y, dtype=torch.float32).view(-1, 1)
    test_x, test_y = torch.tensor(test_x, dtype=torch.float32), torch.tensor(test_y, dtype=torch.float32).view(-1, 1)

    # Create DataLoaders
    train_loader = DataLoader(TensorDataset(train_x, train_y), batch_size=64, shuffle=True)
    valid_loader = DataLoader(TensorDataset(valid_x, valid_y), batch_size=64, shuffle=False)

    # Initialize model
    model = ANNModel(train_x.shape[1], mean, var).to(device)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=params["lr"], momentum=params["momentum"])

    # Training loop with MLflow tracking
    with mlflow.start_run(nested=True):
        for epoch in range(epochs):
            model.train()
            running_loss = 0.0
            for batch_x, batch_y in train_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                optimizer.zero_grad()
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()  


            train_loss = running_loss/len(train_loader)
            print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss:.4f}")
                    # Log metrics with step=epoch (important for graphs)
            mlflow.log_metric("train_loss", train_loss, step=epoch)


        # Evaluate on validation set
        model.eval()
        total_loss = 0.0
        with torch.no_grad():
            for batch_x, batch_y in valid_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                total_loss += loss.item()
        
        eval_rmse = (total_loss / len(valid_loader)) ** 0.5
        
        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)
        
        # Save model using MLflow
        # Example input tensor (a batch of zeros with the same shape as one input sample)
        input_example = torch.zeros(1, train_x.shape[1], dtype=torch.float32).numpy()
        mlflow.pytorch.log_model(model, "model", input_example=input_example)
        
        return {"loss": eval_rmse, "status": "SUCCESS", "model": model}


In [6]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return {"loss": result["loss"], "status": STATUS_OK, "model":result["model"]}

In [7]:
# objective({"lr":1e-1,"momentum":0.5})

In [8]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [10]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparamter search using hyperopt
    # Run the optimization
    trials = Trials()
    best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=3, trials=trials)

    # Sort trials based on the loss (lower is better)
    best_trial = sorted(trials.results, key=lambda x: x['loss'])[0]  # Best trial
    best_model = best_trial['model']  # Extract the best model

    mlflow.log_params(best_params)
    mlflow.pytorch.log_model(best_model, 'best_model', signature=singature)

    # Print out the best params and loss
    print("Best hyperparameters:", best_params)
    print("Best eval rmse", best_trial['loss'])

Epoch 1/3, Loss: 29.0480                             
Epoch 2/3, Loss: 21.5140                             
Epoch 3/3, Loss: 15.6919                             
  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]

🏃 View run carefree-cow-92 at: http://127.0.0.1:5000/#/experiments/267861629433437767/runs/99918ec6463a4d3680da4418b2c9b2ab

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/267861629433437767

Epoch 1/3, Loss: 7.8495                                                       
Epoch 2/3, Loss: 1.3019                                                       
Epoch 3/3, Loss: 1.1140                                                       
 33%|███▎      | 1/3 [00:04<00:09,  4.73s/trial, best loss: 3.648903810814486]

🏃 View run sincere-yak-377 at: http://127.0.0.1:5000/#/experiments/267861629433437767/runs/70ccc10c64a243dba177a624ff6be987

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/267861629433437767 

Epoch 1/3, Loss: 23.8440                                                       
Epoch 2/3, Loss: 9.8899                                                        
Epoch 3/3, Loss: 4.2482                                                        
 67%|██████▋   | 2/3 [00:09<00:04,  4.68s/trial, best loss: 1.1260064543753778]

🏃 View run receptive-ray-716 at: http://127.0.0.1:5000/#/experiments/267861629433437767/runs/fdc1cf022a3c41cfb603697899c4ff89

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/267861629433437767  

100%|██████████| 3/3 [00:14<00:00,  4.69s/trial, best loss: 1.1260064543753778]
Best hyperparameters: {'lr': np.float64(0.0045974588038298625), 'momentum': np.float64(0.49524440274275205)}
Best eval rmse 1.1260064543753778
🏃 View run overjoyed-sponge-526 at: http://127.0.0.1:5000/#/experiments/267861629433437767/runs/7b695ab2d3c24dadab4ed407c4b3dbad
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/267861629433437767


In [11]:
import mlflow

model_uri = 'runs:/4c26c217e33347419ca1bdc04094c75a/model'
# This is the input example logged with the model
pyfunc_model = mlflow.pyfunc.load_model(model_uri)
input_data = pyfunc_model.input_example

# Verify the model with the provided input data using the logged dependencies.
# For more details, refer to:
# https://mlflow.org/docs/latest/models.html#validate-models-before-deployment
mlflow.models.predict(
    model_uri=model_uri,
    input_data=input_data,
    env_manager="local",
)

2025/03/30 18:58:04 INFO mlflow.models.python_api: It is highly recommended to use `uv` as the environment manager for predicting with MLflow models as its performance is significantly better than other environment managers. Run `pip install uv` to install uv. See https://docs.astral.sh/uv/getting-started/installation for other installation methods.
2025/03/30 18:58:04 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


{"predictions": [[200.72340393066406]]}